In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)

In [2]:
import scanpy as sc

In [3]:
%config InlineBackend.figure_formats = ['retina']  # increase resolution on retina screens
import numpy as np
import scanpy as sc
import scipy as sci
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle
from scipy import stats
from sklearn.svm import NuSVR


results_file = './write/results/bulk_bulk_pbmc/'

/home/icb/hananeh.aliee/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Functions

In [4]:
import matplotlib.pyplot as plt_
def myPlot_2(orig, result1,result2,legend1, legend2,xlabel,ylabel,title,figDir):
    s1,i1,r1,p1,e1 = sci.stats.linregress(orig,result1)
    s2,i2,r2,p2,e2 = sci.stats.linregress(orig,result2)
    
    print(r1,p1,e1)
    print(r2,p2,e2)

    plt_.scatter(orig,result1, color = 'red', label = str(legend1), s= 25)
    plt_.scatter(orig,result2, color = 'blue', label = str(legend2), s= 25, marker = 'X')

    ymax1 = max(result1)
    ymin1= min(result1)
    
    ymax2 = max(result2)
    ymin2= min(result2)
    
    ymax = max([ymax1,ymax2])
    ymin = min([ymin1,ymin2])
    
    xmax = max(orig)
    xmin= min(orig)

    x = np.linspace(xmin,xmax, 1000)
    y = x
    plt_.plot(x, y , '-.k')

    plt_.xlim(xmin=xmin, xmax=xmax)
    plt_.ylim(ymin=ymin,ymax=ymax)
    #plt_.xlabel(xlabel)
    #plt_.ylabel(ylabel)
    plt_.title(title)
    plt_.legend()
    
    plt_.savefig(figDir, dpi=300)
    
    plt_.show()
    return r1,p1,e1,r2,p2,e2

In [5]:
def max(list_):
    if len(list_)>0:
        m = float(list_[0])
        for x in list_:
            if float(x)>m:
                m = x
        return m

In [6]:
def normalize_proportions(data,copy):
    if copy==True:
        data_copy = data.copy()
    else:
        data_copy = data
    data_copy[data_copy < 0] = 0
    #print(data_copy)
    for raw in data_copy.index:
        sum = data_copy.loc[raw].sum()
        data_copy.loc[raw] = np.divide(data_copy.loc[raw],sum)
    return data_copy

# Read Control Bulks

In [7]:
bulks_dir = './pbmc/cell19/rnaseq/'
data = pd.read_csv(bulks_dir+'GSE107011_Processed_data_TPM.txt', index_col=0, delimiter = '\t')

ERCC_data = data.loc[[x for x in data.index if 'ERCC-' in x]]

data = data.loc[[x for x in data.index if 'ERCC-' not in x]]
index = [x.split(".")[0] for x in data.index]
data.index = index

duplicatedData = data[data.index.duplicated()]
data = data[data.index.duplicated()==False]
data.shape

samples = data.loc[:,[x for x in data.columns if '_PBMC' in x]]
sorted_bulks_raw = data.loc[:,[x for x in data.columns if '_PBMC' not in x]]
cellTypes_29 = [x[5:] for x in sorted_bulks_raw.columns if '9JD4' in x]
sorted_bulks_raw.columns = [x[5:] for x in sorted_bulks_raw.columns]

In [8]:
signatures_cell19 = pd.read_csv(bulks_dir+'sigRNA.txt', index_col=0, delimiter = '\t')
signatures_cell19.shape

(1296, 17)

In [9]:
sorted_bulks = pd.DataFrame(index=sorted_bulks_raw.index, columns = cellTypes_29)
for col in cellTypes_29:
    sorted_bulks.loc[:,col] = np.mean(sorted_bulks_raw.loc[:,col],axis=1)
print(sorted_bulks.shape)
print(sorted_bulks.columns)

(58174, 29)
Index(['CD8_naive', 'CD8_CM', 'CD8_EM', 'CD8_TE', 'MAIT', 'VD2+', 'VD2-',
       'TFH', 'Treg', 'Th1', 'Th1/Th17', 'Th17', 'Th2', 'CD4_naive', 'CD4_TE',
       'Progenitor', 'B_naive', 'B_NSM', 'B_Ex', 'B_SM', 'Plasmablasts',
       'C_mono', 'I_mono', 'NC_mono', 'NK', 'pDC', 'mDC', 'Neutrophils',
       'Basophils'],
      dtype='object')


In [10]:
sorted_bulks_paper = pd.read_csv(bulks_dir+'wholeGenes/ABIS_RNASeq_full.txt', index_col=0, delimiter = '\t')
sorted_bulks_paper.shape

(17487, 17)

In [11]:
signatures_cell19.columns

Index(['Monocytes C', 'NK', 'T CD8 Memory', 'T CD4 Naive', 'T CD8 Naive',
       'B Naive', 'T CD4 Memory', 'MAIT', 'T gd Vd2', 'Neutrophils LD',
       'T gd non-Vd2', 'Basophils LD', 'Monocytes NC+I', 'B Memory', 'mDCs',
       'pDCs', 'Plasmablasts'],
      dtype='object')

In [12]:
annotations = pd.read_csv(bulks_dir+'annotations.txt', index_col=0, delimiter = '\t')

In [13]:
samples = samples.loc[[x for x in samples.index if 'ERCC-' not in x]]
index = [x.split(".")[0] for x in samples.index]
samples.index = index

duplicatedSamples = samples[samples.index.duplicated()]
samples = samples[samples.index.duplicated()==False]
samples.shape

(58174, 13)

In [14]:
sorted_bulks_proc = sorted_bulks.loc[annotations.index].copy()
sorted_bulks_raw_proc = sorted_bulks_raw.loc[annotations.index].copy()
samples_proc = samples.loc[annotations.index].copy()

samples_proc.index = annotations.loc[:,'Gene Symbol']
#sorted_bulks_selection_proc.index = annotations.loc[:,'Gene Symbol']
sorted_bulks_raw_proc.index = annotations.loc[:,'Gene Symbol']

In [15]:
sorted_bulks_proc.index = annotations.loc[:,'Gene Symbol']
sorted_bulks_proc = sorted_bulks_proc[~sorted_bulks_proc.index.duplicated()]


In [16]:
print(sorted_bulks_raw_proc.shape)
sorted_bulks_raw_proc = sorted_bulks_raw_proc[~sorted_bulks_raw_proc.index.duplicated()]
print(sorted_bulks_raw_proc.shape)
samples_proc = samples_proc[~samples_proc.index.duplicated()]

sorted_bulks_paper = sorted_bulks_paper[~sorted_bulks_paper.index.duplicated()]


(58174, 114)
(56470, 114)


In [17]:
shareds = sorted_bulks_paper.index.intersection(sorted_bulks_raw_proc.index)
sorted_bulks_raw_proc = sorted_bulks_raw_proc.loc[shareds,:]
samples_proc = samples_proc.loc[shareds,:]

In [18]:
sorted_bulks_proc = sorted_bulks_proc.loc[shareds,:]

In [23]:
fractions_raw = pd.read_csv(bulks_dir+'wholeGenes/fractions.txt', index_col=0, delimiter = '\t')
cellsToRemove = ['B NSM','B SM', 'B Ex','Tregs','Tfh', 'Th1', 'Th1/Th17', 'Th17','Th2', 'T CD4 TE','T CD8 EM','T CD8 CM', 'T CD8 TE','Monocytes I', 'Monocytes NC']
fractions = fractions_raw.drop(cellsToRemove,axis=1)
fractions['B Memory'] = fractions_raw.loc[:,['B NSM','B SM', 'B Ex']].sum(axis=1)
fractions['T CD4 Memory'] = fractions_raw.loc[:,['Tregs','Tfh', 'Th1', 'Th1/Th17', 'Th17','Th2', 'T CD4 TE']].sum(axis=1)
fractions['T CD8 Memory'] = fractions_raw.loc[:,['T CD8 EM','T CD8 CM', 'T CD8 TE']].sum(axis=1)
fractions['Monocytes NC+I'] = fractions_raw.loc[:,['Monocytes I', 'Monocytes NC']].sum(axis=1)
fractions[signatures_cell19.columns].columns

Index(['Monocytes C', 'NK', 'T CD8 Memory', 'T CD4 Naive', 'T CD8 Naive',
       'B Naive', 'T CD4 Memory', 'MAIT', 'T gd Vd2', 'Neutrophils LD',
       'T gd non-Vd2', 'Basophils LD', 'Monocytes NC+I', 'B Memory', 'mDCs',
       'pDCs', 'Plasmablasts'],
      dtype='object')

In [25]:
sc.set_figure_params(dpi=100, dpi_save=300, vector_friendly=True)
params = {'legend.fontsize': 8,
          'figure.figsize': (3, 3),
         'axes.labelsize':8,
         'axes.titlesize':8,
         'xtick.labelsize':8,
         'ytick.labelsize':8}
mpl.rcParams.update(params)

# Cibersort genes

In [26]:
sorted_bulks_raw_proc.shape

(17487, 114)

In [27]:
bulks_proc = sc.AnnData(sorted_bulks_raw_proc.T)
bulks_proc.obs['cells'] = sorted_bulks_raw.columns.values
bulks_proc.var_names_make_unique()
bulks_proc.obs_names_make_unique()
sc.pp.normalize_per_cell(bulks_proc, counts_per_cell_after=1e4,)
sc.pp.log1p(bulks_proc)
sc.pp.highly_variable_genes(bulks_proc, flavor='seurat', n_top_genes=4000 , n_bins=5)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [28]:
bulks_proc_hv = bulks_proc[:,bulks_proc.var['highly_variable']==True].copy()

In [29]:
sorted_bulks_paper_hv = sorted_bulks_paper.loc[bulks_proc_hv.var_names,:]

In [30]:
sorted_bulks_paper_hv = sorted_bulks_paper_hv.loc[:,['T CD4 Naive','T CD4 Memory','T CD8 Memory','T CD8 Naive','MAIT','T gd Vd2','T gd non-Vd2'
, 'NK','B Memory','B Naive','pDCs','mDCs', 'Monocytes C','Monocytes NC+I', 'Neutrophils LD','Basophils LD','Plasmablasts']]

In [32]:
sorted_bulks_paper_hv.shape

(4000, 17)

# GA calling

In [77]:
def condition_obj(array):
    return LA.cond(array, p=2)

In [81]:
import datetime
print(datetime.datetime.now())
time0 = datetime.datetime.now()
import autogenes as ag
ngenes = 500 #400
ngen=5000
centroids = ag.init(sorted_bulks_paper_hv.T)
ag.optimize(ngen=ngen,seed=0,nfeatures=ngenes,mode='fixed',objectives=[condition_obj],weights=[-1])

print(datetime.datetime.now()-time0)

2020-09-22 14:06:41.136821
gen	nevals	pareto	condition_obj  
0  	100   	1     	196.1 - 4615.56
1  	100   	1     	196.1 - 1088.75
2  	100   	1     	196.1 - 774.04 
3  	100   	1     	58.9 - 513.6   
4  	100   	1     	58.9 - 381.64  
5  	100   	1     	57.76 - 309.31 
6  	100   	1     	57.62 - 276.44 
7  	100   	1     	57.1 - 250.26  
8  	100   	1     	57.1 - 239.85  
9  	100   	1     	57.1 - 206.29  
10 	100   	1     	57.1 - 188.3   
11 	100   	1     	57.1 - 171.68  
12 	100   	1     	56.55 - 144.83 
13 	100   	1     	56.55 - 58.81  
14 	100   	1     	55.54 - 58.54  
15 	100   	1     	55.54 - 57.76  
16 	100   	1     	55.54 - 57.57  
17 	100   	1     	55.53 - 57.43  
18 	100   	1     	55.53 - 57.23  
19 	100   	1     	54.15 - 57.01  
20 	100   	1     	54.07 - 56.64  
21 	100   	1     	53.07 - 56.17  
22 	100   	1     	50.29 - 55.05  
23 	100   	1     	49.62 - 54.36  
24 	100   	1     	46.71 - 53.03  
25 	100   	1     	46.44 - 52.02  
26 	100   	1     	46.4 - 49.46   
27 	100   	1     	46.

241	100   	1     	25.15 - 25.72  
242	100   	1     	25.15 - 25.69  
243	100   	1     	25.03 - 25.67  
244	100   	1     	25.03 - 25.64  
245	100   	1     	25.01 - 25.61  
246	100   	1     	25.01 - 25.56  
247	100   	1     	24.96 - 25.48  
248	100   	1     	24.96 - 25.44  
249	100   	1     	24.96 - 25.42  
250	100   	1     	24.87 - 25.41  
251	100   	1     	24.87 - 25.39  
252	100   	1     	24.87 - 25.37  
253	100   	1     	24.8 - 25.35   
254	100   	1     	24.63 - 25.31  
255	100   	1     	24.63 - 25.23  
256	100   	1     	24.63 - 25.14  
257	100   	1     	24.52 - 25.09  
258	100   	1     	24.52 - 25.06  
259	100   	1     	24.52 - 25.03  
260	100   	1     	24.43 - 24.96  
261	100   	1     	24.43 - 24.89  
262	100   	1     	24.43 - 24.86  
263	100   	1     	24.36 - 24.84  
264	100   	1     	24.36 - 24.82  
265	100   	1     	24.36 - 24.76  
266	100   	1     	24.29 - 24.72  
267	100   	1     	24.29 - 24.7   
268	100   	1     	24.29 - 24.69  
269	100   	1     	24.29 - 24.67  
270	100   	1  

482	100   	1     	17.97 - 18.3   
483	100   	1     	17.93 - 18.26  
484	100   	1     	17.93 - 18.24  
485	100   	1     	17.93 - 18.21  
486	100   	1     	17.93 - 18.18  
487	100   	1     	17.93 - 18.14  
488	100   	1     	17.93 - 18.06  
489	100   	1     	17.86 - 18.04  
490	100   	1     	17.86 - 18.03  
491	100   	1     	17.86 - 18.01  
492	100   	1     	17.84 - 18.01  
493	100   	1     	17.81 - 18.0   
494	100   	1     	17.81 - 17.99  
495	100   	1     	17.81 - 17.98  
496	100   	1     	17.81 - 17.97  
497	100   	1     	17.81 - 17.97  
498	100   	1     	17.81 - 17.95  
499	100   	1     	17.81 - 17.94  
500	100   	1     	17.71 - 17.93  
501	100   	1     	17.71 - 17.92  
502	100   	1     	17.71 - 17.92  
503	100   	1     	17.58 - 17.89  
504	100   	1     	17.58 - 17.88  
505	100   	1     	17.58 - 17.87  
506	100   	1     	17.58 - 17.87  
507	100   	1     	17.58 - 17.86  
508	100   	1     	17.58 - 17.85  
509	100   	1     	17.58 - 17.83  
510	100   	1     	17.57 - 17.82  
511	100   	1  

723	100   	1     	14.23 - 14.38  
724	100   	1     	14.19 - 14.37  
725	100   	1     	14.19 - 14.36  
726	100   	1     	14.17 - 14.35  
727	100   	1     	14.17 - 14.34  
728	100   	1     	14.17 - 14.33  
729	100   	1     	14.17 - 14.32  
730	100   	1     	14.04 - 14.31  
731	100   	1     	14.04 - 14.3   
732	100   	1     	14.04 - 14.28  
733	100   	1     	14.04 - 14.27  
734	100   	1     	14.03 - 14.26  
735	100   	1     	14.03 - 14.24  
736	100   	1     	14.03 - 14.23  
737	100   	1     	14.03 - 14.23  
738	100   	1     	14.03 - 14.23  
739	100   	1     	14.03 - 14.23  
740	100   	1     	14.03 - 14.22  
741	100   	1     	14.03 - 14.21  
742	100   	1     	14.02 - 14.21  
743	100   	1     	14.02 - 14.2   
744	100   	1     	13.97 - 14.19  
745	100   	1     	13.97 - 14.18  
746	100   	1     	13.97 - 14.18  
747	100   	1     	13.97 - 14.17  
748	100   	1     	13.97 - 14.16  
749	100   	1     	13.97 - 14.15  
750	100   	1     	13.97 - 14.14  
751	100   	1     	13.97 - 14.13  
752	100   	1  

964	100   	1     	12.1 - 12.23   
965	100   	1     	12.1 - 12.22   
966	100   	1     	12.09 - 12.22  
967	100   	1     	12.09 - 12.21  
968	100   	1     	12.09 - 12.2   
969	100   	1     	12.09 - 12.2   
970	100   	1     	12.09 - 12.19  
971	100   	1     	12.07 - 12.18  
972	100   	1     	12.06 - 12.17  
973	100   	1     	12.06 - 12.16  
974	100   	1     	12.06 - 12.16  
975	100   	1     	12.05 - 12.15  
976	100   	1     	12.05 - 12.13  
977	100   	1     	12.05 - 12.12  
978	100   	1     	12.01 - 12.12  
979	100   	1     	12.01 - 12.12  
980	100   	1     	12.01 - 12.11  
981	100   	1     	12.01 - 12.11  
982	100   	1     	12.01 - 12.1   
983	100   	1     	12.01 - 12.1   
984	100   	1     	12.01 - 12.1   
985	100   	1     	11.99 - 12.09  
986	100   	1     	11.99 - 12.09  
987	100   	1     	11.99 - 12.08  
988	100   	1     	11.99 - 12.07  
989	100   	1     	11.99 - 12.07  
990	100   	1     	11.99 - 12.06  
991	100   	1     	11.97 - 12.06  
992	100   	1     	11.97 - 12.05  
993	100   	1  

...

4033	100   	1     	8.24 - 8.24    
4034	100   	1     	8.24 - 8.24    
4035	100   	1     	8.24 - 8.24    
4036	100   	1     	8.24 - 8.24    
4037	100   	1     	8.24 - 8.24    
4038	100   	1     	8.24 - 8.24    
4039	100   	1     	8.24 - 8.24    
4040	100   	1     	8.24 - 8.24    
4041	100   	1     	8.24 - 8.24    
4042	100   	1     	8.24 - 8.24    
4043	100   	1     	8.24 - 8.24    
4044	100   	1     	8.24 - 8.24    
4045	100   	1     	8.24 - 8.24    
4046	100   	1     	8.24 - 8.24    
4047	100   	1     	8.24 - 8.24    
4048	100   	1     	8.24 - 8.24    
4049	100   	1     	8.24 - 8.24    
4050	100   	1     	8.24 - 8.24    
4051	100   	1     	8.24 - 8.24    
4052	100   	1     	8.24 - 8.24    
4053	100   	1     	8.24 - 8.24    
4054	100   	1     	8.23 - 8.24    
4055	100   	1     	8.23 - 8.24    
4056	100   	1     	8.23 - 8.24    
4057	100   	1     	8.23 - 8.24    
4058	100   	1     	8.23 - 8.24    
4059	100   	1     	8.23 - 8.24    
4060	100   	1     	8.23 - 8.24    
4061	100   	1     	8

4268	100   	1     	8.22 - 8.22    
4269	100   	1     	8.22 - 8.22    
4270	100   	1     	8.22 - 8.22    
4271	100   	1     	8.22 - 8.22    
4272	100   	1     	8.22 - 8.22    
4273	100   	1     	8.22 - 8.22    
4274	100   	1     	8.22 - 8.22    
4275	100   	1     	8.22 - 8.22    
4276	100   	1     	8.22 - 8.22    
4277	100   	1     	8.22 - 8.22    
4278	100   	1     	8.22 - 8.22    
4279	100   	1     	8.22 - 8.22    
4280	100   	1     	8.22 - 8.22    
4281	100   	1     	8.22 - 8.22    
4282	100   	1     	8.22 - 8.22    
4283	100   	1     	8.22 - 8.22    
4284	100   	1     	8.22 - 8.22    
4285	100   	1     	8.22 - 8.22    
4286	100   	1     	8.22 - 8.22    
4287	100   	1     	8.22 - 8.22    
4288	100   	1     	8.21 - 8.22    
4289	100   	1     	8.21 - 8.22    
4290	100   	1     	8.21 - 8.22    
4291	100   	1     	8.21 - 8.22    
4292	100   	1     	8.21 - 8.22    
4293	100   	1     	8.21 - 8.22    
4294	100   	1     	8.21 - 8.22    
4295	100   	1     	8.21 - 8.22    
4296	100   	1     	8

4505	100   	1     	8.19 - 8.2     
4506	100   	1     	8.19 - 8.2     
4507	100   	1     	8.19 - 8.2     
4508	100   	1     	8.19 - 8.2     
4509	100   	1     	8.19 - 8.2     
4510	100   	1     	8.19 - 8.2     
4511	100   	1     	8.19 - 8.2     
4512	100   	1     	8.19 - 8.2     
4513	100   	1     	8.19 - 8.2     
4514	100   	1     	8.19 - 8.2     
4515	100   	1     	8.19 - 8.2     
4516	100   	1     	8.19 - 8.2     
4517	100   	1     	8.19 - 8.2     
4518	100   	1     	8.19 - 8.2     
4519	100   	1     	8.19 - 8.2     
4520	100   	1     	8.19 - 8.2     
4521	100   	1     	8.19 - 8.2     
4522	100   	1     	8.19 - 8.2     
4523	100   	1     	8.19 - 8.2     
4524	100   	1     	8.19 - 8.2     
4525	100   	1     	8.19 - 8.2     
4526	100   	1     	8.19 - 8.2     
4527	100   	1     	8.19 - 8.2     
4528	100   	1     	8.19 - 8.2     
4529	100   	1     	8.19 - 8.2     
4530	100   	1     	8.19 - 8.2     
4531	100   	1     	8.19 - 8.2     
4532	100   	1     	8.19 - 8.19    
4533	100   	1     	8

4742	100   	1     	8.18 - 8.18    
4743	100   	1     	8.18 - 8.18    
4744	100   	1     	8.18 - 8.18    
4745	100   	1     	8.18 - 8.18    
4746	100   	1     	8.18 - 8.18    
4747	100   	1     	8.18 - 8.18    
4748	100   	1     	8.18 - 8.18    
4749	100   	1     	8.18 - 8.18    
4750	100   	1     	8.18 - 8.18    
4751	100   	1     	8.18 - 8.18    
4752	100   	1     	8.18 - 8.18    
4753	100   	1     	8.18 - 8.18    
4754	100   	1     	8.18 - 8.18    
4755	100   	1     	8.18 - 8.18    
4756	100   	1     	8.18 - 8.18    
4757	100   	1     	8.18 - 8.18    
4758	100   	1     	8.18 - 8.18    
4759	100   	1     	8.18 - 8.18    
4760	100   	1     	8.18 - 8.18    
4761	100   	1     	8.18 - 8.18    
4762	100   	1     	8.18 - 8.18    
4763	100   	1     	8.18 - 8.18    
4764	100   	1     	8.18 - 8.18    
4765	100   	1     	8.18 - 8.18    
4766	100   	1     	8.18 - 8.18    
4767	100   	1     	8.18 - 8.18    
4768	100   	1     	8.18 - 8.18    
4769	100   	1     	8.18 - 8.18    
4770	100   	1     	8

4977	100   	1     	8.17 - 8.17    
4978	100   	1     	8.17 - 8.17    
4979	100   	1     	8.17 - 8.17    
4980	100   	1     	8.16 - 8.17    
4981	100   	1     	8.16 - 8.17    
4982	100   	1     	8.16 - 8.17    
4983	100   	1     	8.16 - 8.17    
4984	100   	1     	8.16 - 8.17    
4985	100   	1     	8.16 - 8.17    
4986	100   	1     	8.16 - 8.17    
4987	100   	1     	8.16 - 8.17    
4988	100   	1     	8.16 - 8.17    
4989	100   	1     	8.16 - 8.17    
4990	100   	1     	8.16 - 8.17    
4991	100   	1     	8.16 - 8.17    
4992	100   	1     	8.16 - 8.17    
4993	100   	1     	8.16 - 8.17    
4994	100   	1     	8.16 - 8.17    
4995	100   	1     	8.16 - 8.17    
4996	100   	1     	8.16 - 8.17    
4997	100   	1     	8.16 - 8.17    
4998	100   	1     	8.16 - 8.17    
4999	100   	1     	8.16 - 8.17    
5000	100   	1     	8.16 - 8.17    
0:08:23.764869


In [82]:
from numpy import linalg as LA
def condition(df):
    return LA.cond(df.T, p=2)

In [84]:
pareto=0
c = []
while True:
    sorted_bulks_GA = sorted_bulks_paper_hv[ag.select(index=pareto)]
    #print(condition(centroids_sc_pareto))
    c.append(condition(sorted_bulks_GA))
    pareto = pareto+1
print(c)

ValueError: Invalid index

In [85]:
c

[8.163467754843868]